In [5]:
import networkx as ntx

In [ ]:
import sys
from ruamel.yaml import YAML

In [57]:
import numpy as np

In [39]:
from abc import ABC, abstractmethod

In [29]:
y = YAML()

In [7]:
from collections import namedtuple

In [12]:
System = namedtuple("System", ["producers", "components", "links"])

In [ ]:
Producer = namedtuple("Producer", ["messages", "state"])

In [ ]:
Component = namedtuple("Component", ["transforms", "decisions", "state"])

In [ ]:
Link = namedtuple("Link", ["transport", "state"])

In [ ]:
Message = namedtuple("Message", ["fields", "properties"])

In [13]:
blank_system = System([Producer([], []),],
                       [Component([],[],[],[]),],
                       [Link([], []),])

In [15]:
blank_system.producers

[Producer(fields=[], properties=[])]

In [16]:
import pint

In [17]:
ureg = pint.UnitRegistry()

In [19]:
bps = 1e6 * ureg("byte/second")

In [20]:
time = 1e-3 * ureg.second

In [21]:
data = bps * time

In [22]:
data

<Quantity(1000.0, 'byte')>

In [25]:
image_sensor = Producer({"image data": 1e6 * ureg.byte,
                         "sample rate": 1e3 * ureg.Hz,},
                        {"x": 0.0,
                         "y": 0.0,})

In [27]:
thermocouple = Producer({"image data": 1e6 * ureg.byte,
                         "sample rate": 1e3 * ureg.Hz,},
                         {})

In [ ]:
merge = Component({"merge"})

In [ ]:
blank_system = System([Producer([], []),],
                       [Component([],[],[],[]),],
                       [Link([], []),])

In [41]:
from functools import reduce

In [47]:
"image data" in thermocouple.fields.keys()

True

In [63]:
from typing import Callable

In [66]:
"""
Abstract class which defines the template for component transforms.
These transforms take set input fields of incoming messages and create a new message field.
These transforms can impart a set of properties (power consumption, etc) on the host component.
"""
class Transform(ABC):
    def __init__(self, fields: list[str], properties: list[str], transform: Callable[..., list[dict, dict]]):
        self.fields = fields
        self.properties = properties
        self.transform = transform

    def __call__(self, messages: Message):
        field_chk = np.all([f in messages.fields.keys() for f in self.fields])
        assert field_chk, "Transform's input field(s) not found in incoming messages"

        fields = [messages.fields[f] for f in self.fields]
        new_fields, new_properties = self.transform(*fields)
        return new_fields, new_properties



In [67]:
class Convolve(Transform):
    def __init__(self, fields, properties, kernel_x, kernel_y, filters):
        def cross_corr(size: int):
            x, y = np.sqrt(size)
            ops = (x - kernel_x) * (y - kernel_y) * filters

            properties = {"operations": ops,}
            state = {}
            return properties, state

        super().__init__(fields, properties, cross_corr)

In [ ]:
ntx.DiGraph()